In [1]:
import mlxtend
import numpy as np
import pandas as pd

In [2]:
data = np.array([
   ['빅맥', '감자튀김', '콜라'],  
   ['빅맥', '콜라'],
   ['베토디', '감자튀김', '콜라'],
   ['빅맥', '치즈스틱', '콜라'],
   ['베토디', '감자튀김', '치즈스틱'],  
   ['베토디', '감자튀김', '콜라'],
   ['베토디', '콜라'],
   ['치즈스틱'] 
])

C:\Users\gunso\anaconda3\lib\site-packages\ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


In [3]:
# data를 matrix로 변형
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)

In [4]:
df = pd.DataFrame(te_ary, columns = te.columns_)
df

,감자튀김,베토디,빅맥,치즈스틱,콜라
0,True,False,True,False,True
1,False,False,True,False,True
2,True,True,False,False,True
3,False,False,True,True,True
4,True,True,False,True,False
5,True,True,False,False,True
6,False,True,False,False,True
7,False,False,False,True,False


# Apriori Algorithm

1. K개의 item을 가지고 단일 항목 집단 생성
    * 5개의 item: 감튀, 베토디, 빅맥, 치즈스틱, 콜라
2. 단일 항목 집단에서 최소 지지도 이상의 항목만 선택
    * 최소 지지도 0.3로 설정
3. 2에서 선택된 항목만을 대상으로 2개 항목 집단 생성
4. 2개 항목 집단에서 최소 지지도 이상의 항목만 선택

In [12]:
%%time
from mlxtend.frequent_patterns import apriori

# 최소 지지도 0.3로 설정
# use_colnames=True: itemsets 이름이 column 이름으로 나옴
apriori(df, min_support= 0.3, use_colnames = True) 

Wall time: 5.98 ms


,support,itemsets
0,0.500,(감자튀김)
1,0.500,(베토디)
2,0.375,(빅맥)
3,0.375,(치즈스틱)
4,0.750,(콜라)
5,0.375,"(베토디, 감자튀김)"
6,0.375,"(감자튀김, 콜라)"
7,0.375,"(베토디, 콜라)"
8,0.375,"(콜라, 빅맥)"


In [6]:
from mlxtend.frequent_patterns import association_rules

association_rules(apriori(df, min_support=0.3, use_colnames=True), metric = 'confidence', min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(베토디),(감자튀김),0.500,0.50,0.375,0.75,1.500000,0.12500,2.0
1,(감자튀김),(베토디),0.500,0.50,0.375,0.75,1.500000,0.12500,2.0
2,(감자튀김),(콜라),0.500,0.75,0.375,0.75,1.000000,0.00000,1.0
3,(베토디),(콜라),0.500,0.75,0.375,0.75,1.000000,0.00000,1.0
4,(빅맥),(콜라),0.375,0.75,0.375,1.00,1.333333,0.09375,inf


# FP-Growth Algoritm

In [13]:
from mlxtend.frequent_patterns import fpgrowth

fpgrowth(df, min_support=0.1, use_colnames=True)

,support,itemsets
0,0.750,(콜라)
1,0.500,(감자튀김)
2,0.375,(빅맥)
3,0.500,(베토디)
4,0.375,(치즈스틱)
5,0.375,"(감자튀김, 콜라)"
6,0.375,"(베토디, 감자튀김)"
7,0.250,"(베토디, 감자튀김, 콜라)"
8,0.375,"(콜라, 빅맥)"
9,0.125,"(감자튀김, 빅맥)"


In [8]:
from mlxtend.frequent_patterns import association_rules

association_rules(fpgrowth(df, min_support=0.3, use_colnames=True), metric = 'confidence', min_threshold=0.5, support_only=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(감자튀김),(콜라),0.500,0.750,0.375,0.75,1.000000,0.00000,1.00
1,(콜라),(감자튀김),0.750,0.500,0.375,0.50,1.000000,0.00000,1.00
2,(베토디),(감자튀김),0.500,0.500,0.375,0.75,1.500000,0.12500,2.00
3,(감자튀김),(베토디),0.500,0.500,0.375,0.75,1.500000,0.12500,2.00
4,(콜라),(빅맥),0.750,0.375,0.375,0.50,1.333333,0.09375,1.25
5,(빅맥),(콜라),0.375,0.750,0.375,1.00,1.333333,0.09375,inf
6,(베토디),(콜라),0.500,0.750,0.375,0.75,1.000000,0.00000,1.00
7,(콜라),(베토디),0.750,0.500,0.375,0.50,1.000000,0.00000,1.00
